In [30]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split

In [2]:
test_df = pd.read_csv("smsspam_test.csv",encoding='utf-8')
test_df.head()

,000,10,100,1000,10p,11,12hrs,150,150p,150ppm,...,xxx,ya,yeah,year,years,yes,yesterday,yo,yup,_label_
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
test_df['_label_'].head()

0    0
1    0
2    0
3    1
4    0
Name: _label_, dtype: int64

In [4]:
test_array = test_df['_label_'].values

In [5]:
train_df = pd.read_csv("smsspam_train.csv",encoding='utf-8')
train_df.head()

,000,10,100,1000,10p,11,12hrs,150,150p,150ppm,...,xxx,ya,yeah,year,years,yes,yesterday,yo,yup,_label_
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
train_df['_label_'].head()

0    0
1    0
2    0
3    0
4    0
Name: _label_, dtype: int64

In [7]:
train_array = train_df['_label_'].values

In [25]:
#bnb = BernoulliNB(binarize=0.0)

In [9]:
#bnb.fit(test_array, train_array)

In [10]:
test_array

array([0, 0, 0, ..., 0, 0, 0])

In [72]:
features = train_df.iloc[:,[0,499]].values

In [73]:
features

array([[0, 0],
       [0, 0],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]])

In [74]:
target = train_df.iloc[:,500].values
target

array([0, 0, 0, ..., 0, 1, 0])

In [67]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [44]:
targetX_train = sc.fit_transform(targetX_train)

In [75]:
targetX_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.33, random_state = 10)

In [76]:
classifier = BernoulliNB()

In [77]:
classifier.fit(targetX_train, y_train)

BernoulliNB()

In [82]:
#X_test = sc.transform(X_test)

In [80]:
y_pred = classifier.predict(X_test)

In [81]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])